In [1]:
#importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error
from math import sqrt
import warnings          
from scipy.optimize import minimize
from sklearn.model_selection import TimeSeriesSplit
import itertools
warnings.filterwarnings('ignore')

In [61]:
#reading data set and converting string date column into datetime
df = pd.read_csv('../../Data-Sets/female_birth_ts.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date',inplace=True)

In [11]:
df.head()

,Date,Births
0,1959-01-01,35
1,1959-01-02,32
2,1959-01-03,30
3,1959-01-04,31
4,1959-01-05,44


In [ ]:
##train-testsplit
train=df[0:325].copy()
train.Births=train.Births.astype('double')
test=df[325:].copy()
#Plotting data
train.Births.plot(figsize=(15,8), title= 'Births', fontsize=14)
test.Births.plot(figsize=(15,8), title= 'Births', fontsize=14)
plt.show()

### __Naive Method__

In [ ]:
#last value of the dataframe
print('Last day of train data set:',train.Births.iloc[-1])
test['yhat_naive'] = train.Births.iloc[-1]
print('mse:',mean_squared_error(test.Births, test.yhat_naive))
print('mae:',mean_absolute_error(test.Births, test.yhat_naive))

In [ ]:
#Plotting data
train.Births.plot(figsize=(15,8), title= 'Births', fontsize=14)
test.Births.plot(figsize=(15,8), title= 'Births', fontsize=14)
test.yhat_naive.plot(figsize=(15,8), title= 'Births', fontsize=14)
plt.show()

### __Simple Average__

In [ ]:
#calculating the average of the data set
print('Average of the train data set:',round(train.Births.mean()))
test['yhat_simple_average'] = round(train.Births.mean())
print('mse:',mean_squared_error(test.Births, test.yhat_simple_average))
print('mae:',mean_absolute_error(test.Births, test.yhat_simple_average))

In [ ]:
#Plotting data
train.Births.plot(figsize=(15,8), title= 'Births', fontsize=14)
test.Births.plot(figsize=(15,8), title= 'Births', fontsize=14)
test.yhat_naive.plot(figsize=(15,8), title= 'Births', fontsize=14)
test.yhat_simple_average.plot(figsize=(15,8), title= 'Births', fontsize=14)
plt.show()

### __Moving Average Method__

In [ ]:
def rolling_mean(series=None, window=None):
    #calculating rolling mean and visualising with the original series

    rolling_mean = series.rolling(window=window).mean()

    plt.figure(figsize=(15,5))
    plt.title("Moving average\n window size = {}".format(window))
    plt.plot(rolling_mean, "g", label="Rolling mean trend")

    plt.plot(series[window:], label="Actual values")
    plt.legend(loc="upper left")
    plt.grid(True)

In [ ]:
rolling_mean(df,20)

In [ ]:
print('Rolling mean of the last n day train data set:',train.Births.rolling(window=5).mean().iloc[-1])
test['yhat_moving_average'] = train.Births.rolling(window=5).mean().iloc[-1]
print('mse:',mean_squared_error(test.Births, test.yhat_moving_average))
print('mae:',mean_absolute_error(test.Births, test.yhat_moving_average))

In [ ]:
#Plotting data
train.Births.plot(figsize=(15,8), title= 'Births', fontsize=14)
test.Births.plot(figsize=(15,8), title= 'Births', fontsize=14)
test.yhat_naive.plot(figsize=(15,8), title= 'Births', fontsize=14)
test.yhat_simple_average.plot(figsize=(15,8), title= 'Births', fontsize=14)
test.yhat_moving_average.plot(figsize=(15,8), title= 'Births', fontsize=14)
plt.show()

### __Decomposition__

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 18, 8

decomposition = sm.tsa.seasonal_decompose(df.Births, model='additive')
fig = decomposition.plot()
plt.show()

## __Explonential Smoothing__

### __Simple Explonential Smoothing__

In [ ]:
ses = SimpleExpSmoothing(train.Births).fit()
print('Explonential smoothing with optimised alpha:',ses.forecast(1).iloc[0])
test['yhat_simple_explonential_smoothing'] = ses.forecast(1).iloc[0]
print('mse:',mean_squared_error(test.Births, test.yhat_simple_explonential_smoothing))
print('mae:',mean_absolute_error(test.Births, test.yhat_simple_explonential_smoothing))

In [ ]:
#Plotting data
train.Births.plot(figsize=(15,8), title= 'Births', fontsize=14, legend=True)
test.Births.plot(figsize=(15,8), title= 'Births', fontsize=14, legend=True)
test.yhat_simple_explonential_smoothing.plot(figsize=(15,8), title= 'Births', fontsize=14, legend=True)
ses.fittedvalues.plot(marker='o', color='green')
plt.show()

### __Holt's Linear Trend__

In [ ]:
holt_ = Holt(train.Births).fit()
test['yhat_holts_linear_trend'] = holt_.forecast(len(test))
print('mse:',mean_squared_error(test.Births, test.yhat_holts_linear_trend))
print('mae:',mean_absolute_error(test.Births, test.yhat_holts_linear_trend))

In [ ]:
#Plotting data
train.Births.plot(figsize=(15,8), title= 'Births', fontsize=14, legend=True)
test.Births.plot(figsize=(15,8), title= 'Births', fontsize=14, legend=True)
test.yhat_holts_linear_trend.plot(figsize=(15,8), title= 'Births', fontsize=14, legend=True)
holt_.fittedvalues.plot(marker='o', color='green')
plt.show()

In [ ]:
holt_.params

### __Holt-Winter's Seasonal Method__

In [ ]:
holt_winter_1 = ExponentialSmoothing(train.Births,trend='add', seasonal='add', seasonal_periods=7)
holt_winter_1 = holt_winter_1.fit(optimized=True,use_basinhopping=True)
test['yhat_holt_winter_1'] = holt_winter_1.forecast(len(test))
print('mse:',mean_squared_error(test.Births, test.yhat_holt_winter_1))
print('mae:',mean_absolute_error(test.Births, test.yhat_holt_winter_1))

In [ ]:
#Plotting data
train.Births.plot(figsize=(15,8), title= 'Births', fontsize=14, legend=True)
test.Births.plot(figsize=(15,8), title= 'Births', fontsize=14, legend=True)
test.yhat_holt_winter_1.plot(figsize=(15,8), title= 'Births', fontsize=14, legend=True)
holt_winter_1.fittedvalues.plot(marker='o', color='green')
plt.show()

In [ ]:
holt_winter_1.params

In [ ]:
holt_winter_1.level.plot(title='Level')
plt.figure()
holt_winter_1.slope.plot(title='Slope')
plt.figure()
holt_winter_1.season.plot(title='Season')
plt.figure()

### __Cross Validation and Parameter Optimisation__

In [ ]:
class HoltWinters:
    
    """
    Holt-Winters model with the anomalies detection using Brutlag method
    
    # series - initial time series
    # slen - length of a season
    # alpha, beta, gamma - Holt-Winters model coefficients
    # n_preds - predictions horizon
    # scaling_factor - sets the width of the confidence interval by Brutlag (usually takes values from 2 to 3)
    
    """
    
    
    def __init__(self, series, slen, alpha, beta, gamma, n_preds, scaling_factor=1.96):
        self.series = series
        self.slen = slen
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.n_preds = n_preds
        self.scaling_factor = scaling_factor
        
        
    def initial_trend(self):
        sum = 0.0
        for i in range(self.slen):
            sum += float(self.series[i+self.slen] - self.series[i]) / self.slen
        return sum / self.slen  
    
    def initial_seasonal_components(self):
        seasonals = {}
        season_averages = []
        n_seasons = int(len(self.series)/self.slen)
        # let's calculate season averages
        for j in range(n_seasons):
            season_averages.append(sum(self.series[self.slen*j:self.slen*j+self.slen])/float(self.slen))
        # let's calculate initial values
        for i in range(self.slen):
            sum_of_vals_over_avg = 0.0
            for j in range(n_seasons):
                sum_of_vals_over_avg += self.series[self.slen*j+i]-season_averages[j]
            seasonals[i] = sum_of_vals_over_avg/n_seasons
        return seasonals   

          
    def triple_exponential_smoothing(self):
        self.result = []
        self.Smooth = []
        self.Season = []
        self.Trend = []
        self.PredictedDeviation = []
        self.UpperBond = []
        self.LowerBond = []
        
        seasonals = self.initial_seasonal_components()
        
        for i in range(len(self.series)+self.n_preds):
            if i == 0: # components initialization
                smooth = self.series[0]
                trend = self.initial_trend()
                self.result.append(self.series[0])
                self.Smooth.append(smooth)
                self.Trend.append(trend)
                self.Season.append(seasonals[i%self.slen])
                
                self.PredictedDeviation.append(0)
                
                self.UpperBond.append(self.result[0] + 
                                      self.scaling_factor * 
                                      self.PredictedDeviation[0])
                
                self.LowerBond.append(self.result[0] - 
                                      self.scaling_factor * 
                                      self.PredictedDeviation[0])
                continue
                
            if i >= len(self.series): # predicting
                m = i - len(self.series) + 1
                self.result.append((smooth + m*trend) + seasonals[i%self.slen])
                
                # when predicting we increase uncertainty on each step
                self.PredictedDeviation.append(self.PredictedDeviation[-1]*1.01) 
                
            else:
                val = self.series[i]
                last_smooth, smooth = smooth, self.alpha*(val-seasonals[i%self.slen]) + (1-self.alpha)*(smooth+trend)
                trend = self.beta * (smooth-last_smooth) + (1-self.beta)*trend
                seasonals[i%self.slen] = self.gamma*(val-smooth) + (1-self.gamma)*seasonals[i%self.slen]
                self.result.append(smooth+trend+seasonals[i%self.slen])
                
                # Deviation is calculated according to Brutlag algorithm.
                self.PredictedDeviation.append(self.gamma * np.abs(self.series[i] - self.result[i]) 
                                               + (1-self.gamma)*self.PredictedDeviation[-1])
                     
            self.UpperBond.append(self.result[-1] + 
                                  self.scaling_factor * 
                                  self.PredictedDeviation[-1])

            self.LowerBond.append(self.result[-1] - 
                                  self.scaling_factor * 
                                  self.PredictedDeviation[-1])

            self.Smooth.append(smooth)
            self.Trend.append(trend)
            self.Season.append(seasonals[i%self.slen])

In [ ]:
def timeseriesCVscore(params, series, loss_function=mean_squared_error, slen=7):
    """
        Returns error on CV  
        
        params - vector of parameters for optimization
        series - dataset with timeseries
        slen - season length for Holt-Winters model
    """
    # errors array
    errors = []
    
    values = series.values
    alpha, beta, gamma = params
    
    # set the number of folds for cross-validation
    tscv = TimeSeriesSplit(n_splits=3) 
    
    # iterating over folds, train model on each, forecast and calculate error
    for train_, test_ in tscv.split(values):

        model = HoltWinters(series=values[train_], slen=slen, 
                            alpha=alpha, beta=beta, gamma=gamma, n_preds=len(test_))
        model.triple_exponential_smoothing()
        
        predictions = model.result[-len(test_):]
        actual = values[test_]
        error = loss_function(predictions, actual)
        errors.append(error)
        
    return np.mean(np.array(errors))

In [ ]:
%%time
data = df.Births[:-40] # leave some data for testing

# initializing model parameters alpha, beta and gamma
x = [0, 0, 0] 

# Minimizing the loss function 
opt = minimize(timeseriesCVscore, x0=x, 
               args=(data, mean_squared_error), 
               method="TNC", bounds = ((0, 1), (0, 1), (0, 1))
              )

# Take optimal values...
alpha_final, beta_final, gamma_final = opt.x

# ...and train the model with them, forecasting for the next 40 days
model = HoltWinters(data, slen = 7, 
                    alpha = alpha_final, 
                    beta = beta_final, 
                    gamma = gamma_final, 
                    n_preds = 40, scaling_factor = 3)
model.triple_exponential_smoothing()

In [ ]:
def plotHoltWinters(series, plot_intervals=False, plot_anomalies=False):
    """
        series - dataset with timeseries
        plot_intervals - show confidence intervals
        plot_anomalies - show anomalies 
    """
    
    plt.figure(figsize=(20, 10))
    plt.plot(model.result, label = "Model")
    plt.plot(series.values, label = "Actual")
    error = mean_squared_error(series.values, model.result[:len(series)])
    print('mse:',mean_squared_error(series.values, model.result[:len(series)]))
    print('mae:',mean_absolute_error(series.values, model.result[:len(series)]))
    plt.title("Mean Squared Error: {0:.2f}".format(error))
    
    plt.vlines(len(series), ymin=min(model.LowerBond), ymax=max(model.UpperBond), linestyles='dashed')
    plt.axvspan(len(series)-20, len(model.result), alpha=0.3, color='lightgrey')
    plt.grid(True)
    plt.axis('tight')
    plt.legend(loc="best", fontsize=13);
    
plotHoltWinters(df.Births)

### __SARIMA__

In [ ]:
p = d = q = range(0, 5)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 7) for x in list(itertools.product(p, d, q))]

In [ ]:
results = []
best_aic = float("inf")

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(df.Births,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            result = mod.fit()

        except:
            continue
        
        aic = result.aic
        # saving best model, AIC and parameters
        if aic < best_aic:
            best_model = model
            best_aic = aic
            best_param = param
        
        results.append([param, result.aic])

result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'aic']
# sorting in ascending order, the lower AIC is - the better
result_table = result_table.sort_values(by='aic', ascending=True).reset_index(drop=True)


## __Feature Extraction__

In [71]:
df_feature = df.copy()

In [70]:
def lag_features(df=None,y=None,start=None,stop=None):
    for i in range(start, stop):
        df["lag_{}".format(i)] = df[y].shift(i)

In [74]:
def temporal_features(df=None):
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df['weekofyear'] = df.index.weekofyear

In [72]:
lag_creation(df_feature,'Births',1,6)

In [75]:
temporal_features(df_feature)

In [76]:
df_feature

,Births,lag_1,lag_2,lag_3,lag_4,lag_5,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear
Date,,,,,,,,,,,,,,
1959-01-01,35,NaN,NaN,NaN,NaN,NaN,0,3,1,1,1959,1,1,1
1959-01-02,32,35.0,NaN,NaN,NaN,NaN,0,4,1,1,1959,2,2,1
1959-01-03,30,32.0,35.0,NaN,NaN,NaN,0,5,1,1,1959,3,3,1
1959-01-04,31,30.0,32.0,35.0,NaN,NaN,0,6,1,1,1959,4,4,1
1959-01-05,44,31.0,30.0,32.0,35.0,NaN,0,0,1,1,1959,5,5,2
1959-01-06,29,44.0,31.0,30.0,32.0,35.0,0,1,1,1,1959,6,6,2
1959-01-07,45,29.0,44.0,31.0,30.0,32.0,0,2,1,1,1959,7,7,2
1959-01-08,43,45.0,29.0,44.0,31.0,30.0,0,3,1,1,1959,8,8,2
1959-01-09,38,43.0,45.0,29.0,44.0,31.0,0,4,1,1,1959,9,9,2


In [30]:
df_feature["hour"] = df_feature.index.hour
df_feature["weekday"] = df_feature.index.weekday
df_feature['is_weekend'] = df_feature.weekday.isin([5,6])*1

In [38]:
df_feature['Births_rolling_mean_3'] = df_feature['lag_1'].rolling(3).mean()
df_feature['Births_rolling_median_3'] = df_feature['lag_1'].rolling(3).median()
df_feature['Births_rolling_min_3'] = df_feature['lag_1'].rolling(3).min()
df_feature['Births_rolling_max_3'] = df_feature['lag_1'].rolling(3).max()
df_feature['Births_rolling_std_3'] = df_feature['lag_1'].rolling(3).std()

In [39]:
df_feature

,Births,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,hour,weekday,is_weekend,Births_rolling_mean_3,Births_rolling_median_3,Births_rolling_min_3,Births_rolling_max_3,Births_rolling_std_3
Date,,,,,,,,,,,,,,,,,,
1959-01-01,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,3,0,NaN,NaN,NaN,NaN,NaN
1959-01-02,32,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,4,0,NaN,NaN,NaN,NaN,NaN
1959-01-03,30,32.0,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,5,1,NaN,NaN,NaN,NaN,NaN
1959-01-04,31,30.0,32.0,35.0,NaN,NaN,NaN,NaN,NaN,NaN,0,6,1,32.333333,32.0,30.0,35.0,2.516611
1959-01-05,44,31.0,30.0,32.0,35.0,NaN,NaN,NaN,NaN,NaN,0,0,0,31.000000,31.0,30.0,32.0,1.000000
1959-01-06,29,44.0,31.0,30.0,32.0,35.0,NaN,NaN,NaN,NaN,0,1,0,35.000000,31.0,30.0,44.0,7.810250
1959-01-07,45,29.0,44.0,31.0,30.0,32.0,35.0,NaN,NaN,NaN,0,2,0,34.666667,31.0,29.0,44.0,8.144528
1959-01-08,43,45.0,29.0,44.0,31.0,30.0,32.0,35.0,NaN,NaN,0,3,0,39.333333,44.0,29.0,45.0,8.962886
1959-01-09,38,43.0,45.0,29.0,44.0,31.0,30.0,32.0,35.0,NaN,0,4,0,39.000000,43.0,29.0,45.0,8.717798
